In [6]:
from pathlib import Path
import pandas as pd
import numpy as np

In [7]:
raw_data_dir = Path().resolve().parent / "_data" / "healthy" / "raw"
real_mouse_timepoints = {}
mouse_metadata = pd.read_csv(raw_data_dir / "metadata.tsv", sep='\t')
for mouse in ['2', '3', '4', '5']:
    mouse_times = []
    mouse_slice = mouse_metadata.loc[
        mouse_metadata['sampleID'].str.startswith(f'{mouse}-')
        & (mouse_metadata['time'] != 0.0)
        & (mouse_metadata['time'] != 0.5)
    ]
    real_mouse_timepoints[mouse] = np.sort(mouse_slice['time'])
real_mouse_timepoints

{'2': array([ 1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,  6. ,  7. ,
         8. ,  9. , 10. , 11. , 14. , 16. , 18. , 21. , 21.5, 22. , 22.5,
        23. , 23.5, 24. , 25. , 28. , 28.5, 29. , 29.5, 30. , 30.5, 31. ,
        31.5, 32. , 33. , 35. , 35.5, 36. , 36.5, 37. , 37.5, 38. , 39. ,
        42. , 42.5, 43. , 43.5, 44. , 44.5, 45. , 45.5, 46. , 47. , 50. ,
        50.5, 51. , 51.5, 52. , 52.5, 53. , 54. , 57. , 57.5, 58. , 58.5,
        59. , 59.5, 60. , 60.5, 61. , 62. , 63. , 64. , 64.5]),
 '3': array([ 1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,  6. ,  7. ,
         8. ,  9. , 10. , 11. , 14. , 16. , 18. , 21. , 21.5, 22. , 22.5,
        23. , 23.5, 24. , 25. , 28. , 29. , 29.5, 30. , 30.5, 31. , 31.5,
        32. , 33. , 35. , 35.5, 36. , 36.5, 37. , 38. , 39. , 42. , 42.5,
        43. , 43.5, 44. , 44.5, 45. , 45.5, 46. , 47. , 50. , 50.5, 51. ,
        51.5, 52. , 52.5, 53. , 54. , 57. , 57.5, 58. , 58.5, 59. , 59.5,
        60. , 60.5, 61. , 62. , 63. , 

In [8]:
output_dir = Path().resolve().parent / "_outputs"  # directory containing R output matrices.
target_dir = Path() / "mbtransfer"  # destination directory

In [15]:
def inverse_log(y: float, base: float, eps: float):
    """ Applied the inverse transformation of y=log_10(x+eps) """
    return np.power(base, y) - eps


subdirs = [
    ('1e-4', np.exp(1), 1e-4), 
    ('1e-5', np.exp(1), 1e-5),
    ('1e-6', np.exp(1), 1e-6),
]

for (subdir, base, eps) in subdirs:
    print(f'subdir: {subdir}, base: {base}, eps={eps}')
    df_entries = []
    for mouse_idx, mouse_id in enumerate(['2', '3', '4', '5']):
        mouse_pred = pd.read_csv(output_dir / subdir / f"m{mouse_id}_forecast.csv", index_col=0)
        mouse_input = pd.read_csv(output_dir / subdir / f"m{mouse_id}_ground_truth.csv", index_col=0)
        for t_idx, t in enumerate(real_mouse_timepoints[mouse_id]):
            if np.abs(np.round(t, 0) - t) < 0.01:
                t_integer = int(np.round(t, 0))
            else:
                continue
            for taxon_idx, taxon in enumerate(mouse_pred.index):
                df_entries.append({
                    'HeldoutSubjectId': mouse_id,
                    'HeldoutSubjectIdx': mouse_idx,
                    'Method': 'mbtransfer',
                    'TaxonIdx': taxon_idx,
                    'TimePoint': t_idx,
                    'Truth': inverse_log(mouse_input.loc[taxon, f'm{mouse_id}_T{t_integer}'], base=base, eps=eps),
                    'Pred': inverse_log(mouse_pred.loc[taxon, f'm{mouse_id}_T{t_integer}'], base=base, eps=eps)
                })
                
    heldout_rel_df = pd.DataFrame(df_entries)
    del df_entries
    (target_dir / subdir).mkdir(exist_ok=True, parents=True)
    heldout_rel_df.sort_values(['HeldoutSubjectId', 'TaxonIdx']).to_csv(target_dir / subdir / "relative_cv.tsv", sep='\t', index=False)

subdir: 1e-4, base: 2.718281828459045, eps=0.0001
subdir: 1e-5, base: 2.718281828459045, eps=1e-05
subdir: 1e-6, base: 2.718281828459045, eps=1e-06


In [12]:
np.log(1e-4)

-9.210340371976182

In [14]:
inverse_log(-9.21034037197618, base=np.exp(1), eps=1e-4)

-9.999938388805618e-05